## Simple example of usage of symmetries discretization

This example will walk you through the usage of the discretization function found in `happypose_msgs_py`.
Note, this example requires additional dependency in a form of [MeshCat](https://pypi.org/project/meshcat/) which has to be installed manually.

Additionally, the user has to update `PYTHONPATH` variable used inside Jupyter notebook to account for dependencies found in their ROS 2 installation and build dependencies of `happypose_msgs` build in their Colcon workspace. Code cell will help to make those changes.

In [ ]:
# Add ROS 2 install path and your Colcon workspace to PYTHONPATH in Jupyter
import os
import sys
from pathlib import Path

# Modify this path to mach you Colcon workspace. The path has to be global
my_colcon_ws_path = Path("/home/gepetto/ros2_ws")

python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
dist_package_path = Path("local") / "lib" / python_version / "dist-packages"
ros_path = Path("/opt") / "ros" / os.environ['ROS_DISTRO'] / dist_package_path
colson_ws_path = my_colcon_ws_path / "install" / "happypose_msgs" / dist_package_path
sys.path.append(ros_path.as_posix())
sys.path.append(colson_ws_path.as_posix())

In [ ]:
import time

import meshcat
import meshcat.geometry as g

from geometry_msgs.msg import Vector3
from happypose_msgs_py.symmetries import discretize_symmetries
from happypose_msgs.msg import ContinuousSymmetry, ObjectSymmetries

# Generate input ROS message with symmetries
input_msg = ObjectSymmetries(
    symmetries_discrete=[],
    symmetries_continuous=[
        ContinuousSymmetry(
            axis=Vector3(x=0.0, y=0.0, z=1.0),
            offset=Vector3(x=0.0, y=0.0, z=0.0),
        )
    ],
)

# Discretize symmetries from the message
res = discretize_symmetries(input_msg, n_symmetries_continuous=64)

Create MeshCat window to display simple mesh rotating around our symmetries

In [ ]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

Load mesh of Valkyrie robot head and spin it around our symmetry axis

In [4]:
assests_path = Path(meshcat.viewer_assets_path()) / "data"

vis["robots/valkyrie/head"].set_object(
    g.ObjMeshGeometry.from_file(assests_path / "head_multisense.obj"),
    g.MeshLambertMaterial(
        map=g.ImageTexture(
            image=g.PngImage.from_file(assests_path / "HeadTextureMultisense.png")
        )
    ),
)

for r in res:
    # Apply our symmetry transformation in a form of matrix
    vis["robots/valkyrie/head"].set_transform(r)
    time.sleep(0.1)